In [2]:
# %% [markdown]
# # Clearance Discount Model for Bottom 10 Products
# Uses rule-based tiers + uplift simulation.

import pandas as pd
import numpy as np
from pathlib import Path

IN_DIR = Path("./outputs_low_sellers")
OUT_FILE = IN_DIR / "clearance_recommendations.csv"

feat = pd.read_csv(IN_DIR / "bottom10_features.csv")

# ----- Discount Tier Logic -----
def base_discount(row):
    q = row['Avg_Monthly_Quantity']
    zr = row['Zero_Sales_Ratio']
    if q < 10 and zr >= 0.50: return 0.50
    if q < 20 and zr >= 0.30: return 0.40
    if q < 30 or zr >= 0.20: return 0.30
    if q < 40: return 0.25
    return 0.20

feat['Base_Discount'] = feat.apply(base_discount, axis=1)

# Adjustments
feat['Adj_Discount'] = feat['Base_Discount']

# Last sale > 3 months ago -> +10%
feat.loc[feat['Last_Sale_Months_Ago'] >= 3, 'Adj_Discount'] += 0.10

# Low variability & consistently slow: CV < 0.3 -> +5%
feat.loc[feat['CV_Monthly_Quantity'] < 0.3, 'Adj_Discount'] += 0.05

# Cap at 0.60
feat['Adj_Discount'] = feat['Adj_Discount'].clip(upper=0.60)

# Assume elasticity
ELASTICITY = -1.2

feat['Demand_Multiplier'] = 1 + (abs(ELASTICITY) * feat['Adj_Discount'])
feat['Demand_Multiplier'] = feat['Demand_Multiplier'].clip(upper=2.5)

# Projected units (avoid zero)
feat['Projected_Units'] = np.maximum(feat['Avg_Monthly_Quantity'] * feat['Demand_Multiplier'], feat['Avg_Monthly_Quantity'] + 1)

# Approx average price = Avg_Monthly_Value / Avg_Monthly_Quantity
feat['Avg_Price_Est'] = feat['Avg_Monthly_Value'] / feat['Avg_Monthly_Quantity'].replace(0,np.nan)
feat['Discounted_Price'] = feat['Avg_Price_Est'] * (1 - feat['Adj_Discount'])

feat['Projected_Revenue'] = feat['Projected_Units'] * feat['Discounted_Price']
feat['Current_Revenue'] = feat['Avg_Monthly_Quantity'] * feat['Avg_Price_Est']
feat['Revenue_Lift'] = feat['Projected_Revenue'] - feat['Current_Revenue']

# (Optional) Assume margin 30% pre-discount, adjust margin after discount
ASSUMED_MARGIN = 0.30
feat['Current_Gross_Profit'] = feat['Current_Revenue'] * ASSUMED_MARGIN
feat['Projected_Gross_Profit'] = feat['Projected_Revenue'] * (ASSUMED_MARGIN - feat['Adj_Discount']*0.5).clip(lower=0.05)
feat['Gross_Profit_Delta'] = feat['Projected_Gross_Profit'] - feat['Current_Gross_Profit']

selected_cols = [
    'Description','Avg_Monthly_Quantity','Zero_Sales_Ratio','Last_Sale_Months_Ago',
    'CV_Monthly_Quantity','Base_Discount','Adj_Discount','Demand_Multiplier',
    'Projected_Units','Avg_Price_Est','Discounted_Price','Current_Revenue',
    'Projected_Revenue','Revenue_Lift','Current_Gross_Profit','Projected_Gross_Profit','Gross_Profit_Delta'
]
rec = feat[selected_cols].sort_values('Adj_Discount', ascending=False)
rec.to_csv(OUT_FILE, index=False)
display(rec.head(10))

print("Saved:", OUT_FILE)


,Description,Avg_Monthly_Quantity,Zero_Sales_Ratio,Last_Sale_Months_Ago,CV_Monthly_Quantity,Base_Discount,Adj_Discount,Demand_Multiplier,Projected_Units,Avg_Price_Est,Discounted_Price,Current_Revenue,Projected_Revenue,Revenue_Lift,Current_Gross_Profit,Projected_Gross_Profit,Gross_Profit_Delta
10,RAIN GIRL CHILDS UMBRELLA,1.0,0.923077,12,NaN,0.5,0.6,1.72,2.00,3.25,1.300,3.25,2.60000,-0.65000,0.975,0.130000,-0.845000
8,PINK HEART CHRISTMAS DECORATION,1.0,0.923077,12,NaN,0.5,0.6,1.72,2.00,0.38,0.152,0.38,0.30400,-0.07600,0.114,0.015200,-0.098800
18,SMALL FOLKART CHRISTMAS TREE DEC,1.0,0.923077,7,NaN,0.5,0.6,1.72,2.00,0.42,0.168,0.42,0.33600,-0.08400,0.126,0.016800,-0.109200
16,S/3 ROSE GARDEN SCENT SQU CANDLE,1.0,0.923077,11,NaN,0.5,0.6,1.72,2.00,1.65,0.660,1.65,1.32000,-0.33000,0.495,0.066000,-0.429000
15,RUSSIAN FOLKART STACKING TINS,1.0,0.923077,11,NaN,0.5,0.6,1.72,2.00,4.95,1.980,4.95,3.96000,-0.99000,1.485,0.198000,-1.287000
14,"RUBY GLASS NECKLACE 42""",1.0,0.923077,3,NaN,0.5,0.6,1.72,2.00,7.50,3.000,7.50,6.00000,-1.50000,2.250,0.300000,-1.950000
11,RED HEART CANDY POP LIGHTS,1.0,0.923077,11,NaN,0.5,0.6,1.72,2.00,4.95,1.980,4.95,3.96000,-0.99000,1.485,0.198000,-1.287000
9,PURPLE CRYSTAL DROP EARRINGS,1.0,0.923077,9,NaN,0.5,0.6,1.72,2.00,2.96,1.184,2.96,2.36800,-0.59200,0.888,0.118400,-0.769600
19,UBO-LIGHT TRIOBASE BLUE,1.0,0.923077,10,NaN,0.5,0.6,1.72,2.00,0.85,0.340,0.85,0.68000,-0.17000,0.255,0.034000,-0.221000
6,HAPPY BIRTHDAY GINGER CAT CARD,2.0,0.923077,9,NaN,0.5,0.6,1.72,3.44,0.19,0.076,0.38,0.26144,-0.11856,0.114,0.013072,-0.100928


Saved: outputs_low_sellers\clearance_recommendations.csv
